In [ ]:
import sys
sys.version

In [ ]:
%%writefile BQ_GetQueryResultToLocal.py

## Get Data From BQ

## install pandas-gbq package; command: pip3 install pandas-gbq
import pandas as pd

def Exec_BQ(query, projectid):
    return pd.io.gbq.read_gbq(query, project_id=projectid, index_col=None, col_order=None, reauth=False, \
                              private_key=None)

project_id =  'moonshot-223910'#'project--demo'
query = '''
Select 
  Sex,
  Count(1) as PassengerCount,
  Sum(Age)/Count(1) as AverageAge,
  Sum(Fare)/Count(1) as AverageFare,
  sum(case when Survived == 0 then 1 else 0 end) as PassengerDied,
  sum(case when Survived == 1 then 1 else 0 end) as PassengerSurvived,
From
  TABLE_QUERY([moonshot-223910.BankAccounts], 'table_id like "Test_TitanicTr%"')
Group by
  Sex
'''
# TABLE_QUERY([project--demo.DemoData], 'table_id like "titainc_train"')
  
DF = Exec_BQ(query, project_id)
print('Dataframe Shape :', DF.shape)
display(DF)

In [ ]:
%%writefile BQ_QueryResultUploadToBQ.py
## Save Data in BQ

## Firstly in BQ create a dataset named "UploadDataTest"
destTable = 'UploadDataTest.BasicResultUpload'
project_id =  'moonshot-223910'#'project--demo'
query = '''
Select 
  Sex,
  Count(1) as PassengerCount,
  Sum(Age)/Count(1) as AverageAge,
  Sum(Fare)/Count(1) as AverageFare,
  sum(case when Survived == 0 then 1 else 0 end) as PassengerDied,
  sum(case when Survived == 1 then 1 else 0 end) as PassengerSurvived,
From
  TABLE_QUERY([moonshot-223910.BankAccounts], 'table_id like "Test_TitanicTr%"')
Group by
  Sex
'''
# TABLE_QUERY([project--demo.DemoData], 'table_id like "titainc_train"')
query = ' '.join(query.split('\n'))
## " should be converted to \" around table be like


terminalCommand = r"""bq --project_id {projectId} --nosync query --batch -q --destination_table '{destinationTable}' --allow_large_results "{queryTxt}" """.format(projectId = project_id, destinationTable = destTable, queryTxt = query)
terminalCommand = terminalCommand.replace('like "', 'like \\"').replace('''"\'''', '''\\"\'''') #"\u".join(txt.split('u'))
print(terminalCommand)
## the printed commannd needs to be run from terminal to achieve this follow the steps mentioned  below. (can also proceede  by creating a .sh file)
#Step1. sudo python BQ_QueryResultUploadToBQ.py > commandToRun.txt
#Step2. sudo chmod +x commandToRun.txt > commandToRun.txt.log
#Step3. sudo commandToRun.txt